In [ ]:
import random
import csv
import datetime
import numpy as np
from datetime import datetime, timedelta, date

In [13]:
def random_date(start_year=2021, end_year=2023):
    year = random.randint(start_year, end_year)
    month = random.randint(1, 12)
    day = random.randint(1, 28)  # To avoid issues with leap years and different month lengths

    generated_date = date(year, month, day)
    day_of_week = generated_date.weekday()

    # More bookings during weekends
    if day_of_week < 5:
        if random.random() < 0.5:
            return generated_date
    else:
        if random.random() < 0.7:
            return generated_date

    return random_date(start_year, end_year)


def save_data_to_csv(filename, data, headers):
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(data)

def generate_customers(num_customers, min_age=15, max_age=32, mean_age=23.5, std_dev_age=4):
    customers = []
    for i in range(1, num_customers + 1):
        age = int(np.random.normal(mean_age, std_dev_age))
        age = max(min_age, min(age, max_age))
        customers.append((i, f"Customer {i}", age, 0))
    return customers

def generate_employees(num_employees, location_ids):
    roles = [('Manager', 3), ('DM', 30), ('Receptionist', 4), ('Treasury', 2)]
    flattened_roles = [role for role, count in roles for _ in range(count)]

    if len(flattened_roles) != num_employees:
        raise ValueError("The total number of roles does not match the number of employees")

    random.shuffle(flattened_roles)

    return [(i, f"Employee {i}", flattened_roles[i - 1], random.choice(location_ids), 0) for i in range(1, num_employees + 1)]

def generate_games(num_games, location_ids, genre_ids):
    games = []
    for i in range(num_games):
        min_players = random.randint(1, 4)
        max_players = random.randint(min_players, 6)
        cost_per_player = random.uniform(15, 50)
        games.append((i + 1, f'Game {i + 1}', f'Description for Game {i + 1}', min_players, max_players, random.choice(location_ids), random.choice(genre_ids), cost_per_player))
    return games

def generate_genres(genre_list):
    return [(idx + 1, genre) for idx, genre in enumerate(genre_list)]

def generate_locations(location_list):
    return [(idx + 1, location) for idx, location in enumerate(location_list)]

def generate_bookings(num_bookings, game_list, employee_ids, location_ids):
    bookings = []
    for i in range(1, num_bookings + 1):
        game = random.choice(game_list)
        min_players, max_players = game[3], game[4]
        if min_players > max_players:
            min_players, max_players = max_players, min_players
        employee_id = random.choice(employee_ids)
        location_id = random.choice(location_ids)
        booking_date = random_date()
        num_players = random.randint(min_players, max_players)
        total_cost = game[-1] * num_players
        tip_received = total_cost * random.normalvariate(0.15, 0.03)
        total_cost += tip_received
        bookings.append((i, game[0], employee_id, location_id, booking_date, num_players, total_cost, tip_received))
        employees[employee_id - 1] = tuple(list(employees[employee_id - 1][:-1]) + [employees[employee_id - 1][-1] + tip_received])

    return bookings

def generate_booking_customers(bookings, customer_ids):
    booking_customers = []
    for booking in bookings:
        num_players = booking[5]
        total_cost = booking[6]
        tip_received = booking[7]
        cost_per_player = (total_cost - tip_received) / num_players
        for _ in range(num_players):
            booking_customers.append((booking[0], random.choice(customer_ids), cost_per_player))
    return booking_customers

num_customers = 2530
num_employees = 39
num_games = 150
num_bookings = 6300

genre_list = ['Horror', 'Joy', 'Deduction', 'Campus', 'Hardcore', 'Emotion', 'Mechanism', 'Costume', 'Republic', 'Modern', 'Science Fiction', 'European', 'Japanese']
location_list = ['Chinatown', 'Koreatown']

# Generate data
customers = generate_customers(num_customers)
locations = generate_locations(location_list)
genres = generate_genres(genre_list)
employees = generate_employees(num_employees, [l[0] for l in locations])
games = generate_games(num_games, [l[0] for l in locations], [g[0] for g in genres])
bookings = generate_bookings(num_bookings, games, [e[0] for e in employees], [l[0] for l in locations])
booking_customers = generate_booking_customers(bookings, [c[0] for c in customers])


for bc in booking_customers:
    customer_id = bc[1]
    cost = bc[2]
    customers[customer_id - 1] = (customers[customer_id - 1][0], customers[customer_id - 1][1], customers[customer_id - 1][2], customers[customer_id - 1][3] + cost)

for booking in bookings:
    employee_id = booking[2]
    tip_received = booking[7]
    employees[employee_id - 1] = (employees[employee_id - 1][0], employees[employee_id - 1][1], employees[employee_id - 1][2], employees[employee_id - 1][3], employees[employee_id - 1][4] + tip_received)

# Save data to CSV files
save_data_to_csv('customers.csv', customers, ['customer_id', 'name', 'age', 'sum_cost'])
save_data_to_csv('locations.csv', locations, ['location_id', 'location'])
save_data_to_csv('genres.csv', genres, ['genre_id', 'genre'])
save_data_to_csv('employees.csv', employees, ['employee_id', 'name', 'role', 'location_id', 'sum_tip'])
save_data_to_csv('games.csv', games, ['game_id', 'name', 'description', 'min_players', 'max_players', 'location_id', 'genre_id', 'cost_per_player'])
save_data_to_csv('bookings.csv', bookings, ['booking_id', 'game_id', 'employee_id', 'location_id', 'booking_date', 'num_players', 'total_cost', 'tip_received'])
save_data_to_csv('booking_customers.csv', booking_customers, ['booking_id', 'customer_id', 'cost'])